### LIBRERIAS

In [ ]:
import numpy as np
import pandas as pd
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('stopwords')
from nltk.corpus import stopwords
from datetime import datetime
import re

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Vamos a realizar un análisis de sentimientos en reseñas de Amazon utilizando NLTK y otros paquetes de Python necesarios. Después de cargar el archivo CSV que contiene 34,660 reseñas de varios productos de Amazon, analizaremos el sentimiento de cada reseña y calcularemos varias estadísticas para ayudar en nuestro análisis.

In [ ]:


import os
os.chdir(r"C:\Users\rportatil111\Documents\DeepLearning\Ejercicio2-20250116T080552Z-001\Ejercicio2")

df = pd.read_csv('1429_1.csv')

df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\rportatil111\\Documents\\DeepLearning\\Ejercicio2-20250116T080552Z-001\\Ejercicio2\\.ipynb_checkpoints'

A veces, un solo producto se lista con diferentes nombres dependiendo de las versiones o, incluso, si el producto está siendo vendido por diferentes vendedores, cada uno podría utilizar su propio nombre o descripción, aunque en realidad estén vendiendo el mismo producto.

In [ ]:
# Agrupar por 'id' y obtener los nombres únicos asociados a cada 'id'
result = df.groupby('id')['name'].nunique()

# Filtrar los 'id' que tienen más de un nombre
ids_con_multiple_nombres = result[result > 1]

# Mostrar los 'id' que tienen más de un nombre
print(ids_con_multiple_nombres)

"C:\Users\rportatil111\Documents\DeepLearning\1429_1 csv del ejercicico 2 PLN.csv"
1429_1.csv


id
AV1YnRtnglJLPUi8IJmV     2
AVpe9CMS1cnluZ0-aoC5     3
AVpfBEWcilAPnD_xTGb7     2
AVpfl8cLLJeJML43AE3S    17
AVphPmHuilAPnD_x3E5h    11
AVphgVaX1cnluZ0-DR74     8
AVpjEN4jLJeJML43rpUe     3
AVqVGWLKnnc1JgDc3jF1     2
AVqVGWQDv8e3D1O-ldFr     2
AVqVGZNvQMlgsOJE6eUY     2
AVqkIiKWnnc1JgDc3khH     2
AVqkIj9snnc1JgDc3khU     2
AVsRjfwAU2_QcyX9PHqe     3
Name: name, dtype: int64


In [ ]:
print(df.loc[df['id'] == 'AVpe9CMS1cnluZ0-aoC5', df.columns[1]].unique())

['Amazon Kindle Lighted Leather Cover,,,\r\nAmazon Kindle Lighted Leather Cover,,,'
 'Amazon Kindle Lighted Leather Cover,,,\r\nKindle Keyboard,,,'
 'Kindle Keyboard,,,\r\nKindle Keyboard,,,']


Se ha decidico no incluir los nombres de los productos porque en muchos casos hay más de un nombre para el mismo ID, y algunos no son muy similares como para tomar uno para todos. Es decir, debido a la variabilidad de los nombres asociados a un mismo producto, se ha decidido utilizar solo los IDs para representar los productos en el análisis.

In [ ]:
# Filtramos el dataframe con las columnas más relevantes
df_filtered = df[['id', 'categories', 'reviews.date', 'reviews.rating', 'reviews.text']]

In [ ]:
# Función para preprocesar el texto (las reviews)
def preprocess_text(review, language='english'):
    # Convertir a minúsculas
    review = review.lower()

    # Eliminar caracteres especiales y números
    review = re.sub(r'[^a-zA-ZáéíóúüñçÁÉÍÓÚÜÑÇ\s]', '', review)

    # Eliminar palabras vacías (stopwords)
    stop_words = set(stopwords.words(language))  # Usar la lista de stopwords según el idioma
    review = ' '.join(word for word in review.split() if word not in stop_words)

    return review

In [ ]:
# Función para extraer el MES de la cadena de fecha (esto será importante para la pregunta sobre el inventario de invierno)
def extract_month(date_str):
    try:
        # Intenta analizar con el primer formato
        date_obj = datetime.strptime(date_str, '%Y-%m-%dT%H:%M:%S.%fZ')
    except ValueError:
        try:
            # Si el primer formato falla, intenta analizar con el segundo formato
            date_obj = datetime.strptime(date_str, '%Y-%m-%dT%H:%M:%SZ')
        except ValueError as e:
            return None
    return date_obj.month

In [ ]:
df_processed = df_filtered.copy()
df_processed['reviews.text'] = df_processed['reviews.text'].astype(str)
df_processed['reviews.date'] = df_processed['reviews.date'].astype(str)

# Preprocesamos el texto
df_processed['reviews.text'] = df_processed['reviews.text'].apply(lambda x: preprocess_text(x))

# Extraemos el mes de la columna 'reviews.date'
df_processed['month'] = df_processed['reviews.date'].apply(extract_month)
df_processed.drop('reviews.date', axis=1, inplace=True)

In [ ]:
def sentiment(document):
    vader_analyzer = SentimentIntensityAnalyzer()
    score = vader_analyzer.polarity_scores(document)
    return score

# Aplicamos la funcion de sentimientos a cada fila en la columna 'review_text'
df_sentiments = df_processed.copy()
df_sentiments['sentiments'] = df_sentiments['reviews.text'].astype(str).apply(sentiment)

# Extraemos los valores 'neg', 'neu' y 'pos' de la columna 'sentiments'
df_sentiments['neg'] = df_sentiments['sentiments'].apply(lambda x: x['neg'])
df_sentiments['neu'] = df_sentiments['sentiments'].apply(lambda x: x['neu'])
df_sentiments['pos'] = df_sentiments['sentiments'].apply(lambda x: x['pos'])

In [ ]:
# Eliminamos la columna original 'sentiments' si no la vamos a usar más
df_sentiments.drop('sentiments', axis=1, inplace=True)
df_sentiments

,id,categories,reviews.rating,reviews.text,month,neg,neu,pos
0,AVqkIhwDv8e3D1O-lebb,"Electronics,iPad & Tablets,All Tablets,Fire Ta...",5.0,product far disappointed children love use lik...,1.0,0.137,0.354,0.509
1,AVqkIhwDv8e3D1O-lebb,"Electronics,iPad & Tablets,All Tablets,Fire Ta...",5.0,great beginner experienced person bought gift ...,1.0,0.000,0.272,0.728
2,AVqkIhwDv8e3D1O-lebb,"Electronics,iPad & Tablets,All Tablets,Fire Ta...",5.0,inexpensive tablet use learn step nabi thrille...,1.0,0.000,0.756,0.244
3,AVqkIhwDv8e3D1O-lebb,"Electronics,iPad & Tablets,All Tablets,Fire Ta...",4.0,ive fire hd two weeks love tablet great valuew...,1.0,0.055,0.492,0.453
4,AVqkIhwDv8e3D1O-lebb,"Electronics,iPad & Tablets,All Tablets,Fire Ta...",5.0,bought grand daughter comes visit set user ent...,1.0,0.000,0.759,0.241
...,...,...,...,...,...,...,...,...
34655,AVpfiBlyLJeJML43-4Tp,"Computers/Tablets & Networking,Tablet & eBook ...",3.0,appreciably faster higher amp charger used kin...,9.0,0.124,0.712,0.164
34656,AVpfiBlyLJeJML43-4Tp,"Computers/Tablets & Networking,Tablet & eBook ...",1.0,amazon include charger kindle fact theyre char...,11.0,0.000,0.811,0.189
34657,AVpfiBlyLJeJML43-4Tp,"Computers/Tablets & Networking,Tablet & eBook ...",1.0,love kindle fire really disappointed kindle po...,10.0,0.127,0.700,0.173
34658,AVpfiBlyLJeJML43-4Tp,"Computers/Tablets & Networking,Tablet & eBook ...",1.0,surprised find come type charging cords purcha...,10.0,0.000,0.758,0.242


In [ ]:
# Calculamos estadísticas
statistics_df = df_sentiments.groupby('id').agg({'id': ['count'], 'reviews.rating': ['mean', 'min', 'max'], 'reviews.text': ['count'], 'neg': ['mean'], 'pos': ['mean'], 'neu': ['mean']})

# Después de usar groupby con agg, cada nombre de columna estará compuesto por un par de nombres como (columna, estadística)
# Aplanar columnas de multiíndice
statistics_df.columns = ['_'.join(col).strip() for col in statistics_df.columns.values]

# Renombramos las columnas count
statistics_df.rename(columns={'reviews.rating_count': 'reviews_count'}, inplace=True)

statistics_df

,id_count,reviews.rating_mean,reviews.rating_min,reviews.rating_max,reviews.text_count,neg_mean,pos_mean,neu_mean
id,,,,,,,,
AV1YE_muvKc47QAVgpwE,5056,4.707278,1.0,5.0,5056,0.072376,0.397614,0.530009
AV1YnR7wglJLPUi8IJmi,372,4.424731,1.0,5.0,372,0.049960,0.415476,0.534581
AV1YnRtnglJLPUi8IJmV,3176,4.772355,1.0,5.0,3176,0.036899,0.408558,0.554542
AVpe7AsMilAPnD_xQ78G,19,4.666667,4.0,5.0,19,0.055000,0.247737,0.697316
AVpe8PEVilAPnD_xRYIi,1,NaN,NaN,NaN,1,0.000000,0.451000,0.549000
AVpe9CMS1cnluZ0-aoC5,5,4.000000,3.0,5.0,5,0.047400,0.192000,0.760400
AVpfBEWcilAPnD_xTGb7,19,NaN,NaN,NaN,19,0.078632,0.273579,0.647842
AVpfIfGA1cnluZ0-emyp,73,4.205479,1.0,5.0,73,0.119918,0.242671,0.637397
AVpf_4sUilAPnD_xlwYV,15,3.066667,1.0,5.0,15,0.075933,0.310467,0.613667


Ahora abordaremos el siguiente problema empresarial para Amazon respondiendo las siguientes preguntas. Dado que pueden ser algo subjetivas, describiremos los rangos que hemos aplicado a cada una de ellas:

### 1. ¿Qué productos deben ser mantenidos?

Productos que tienen una calificación promedio de 4 o superior. Esto asegura que solo se conserven productos con calificaciones consistentemente altas, lo que indica satisfacción entre los clientes. Hay productos que aún no tienen muchas compras pero sí comentarios positivos, por lo que la calificación puede verse afectada fácilmente; en estos casos, también hemos incluido que el producto debe tener un alto sentimiento positivo en las reseñas.

In [ ]:
product_kept = statistics_df[((statistics_df['reviews.rating_mean'] >= 4) | (statistics_df['pos_mean'] >= 0.4)) &
                             (statistics_df['pos_mean'] > statistics_df['neg_mean'])]

product_kept

,id_count,reviews.rating_mean,reviews.rating_min,reviews.rating_max,reviews.text_count,neg_mean,pos_mean,neu_mean
id,,,,,,,,
AV1YE_muvKc47QAVgpwE,5056,4.707278,1.0,5.0,5056,0.072376,0.397614,0.530009
AV1YnR7wglJLPUi8IJmi,372,4.424731,1.0,5.0,372,0.049960,0.415476,0.534581
AV1YnRtnglJLPUi8IJmV,3176,4.772355,1.0,5.0,3176,0.036899,0.408558,0.554542
AVpe7AsMilAPnD_xQ78G,19,4.666667,4.0,5.0,19,0.055000,0.247737,0.697316
AVpe8PEVilAPnD_xRYIi,1,NaN,NaN,NaN,1,0.000000,0.451000,0.549000
AVpe9CMS1cnluZ0-aoC5,5,4.000000,3.0,5.0,5,0.047400,0.192000,0.760400
AVpfIfGA1cnluZ0-emyp,73,4.205479,1.0,5.0,73,0.119918,0.242671,0.637397
AVpff7_VilAPnD_xc1E_,6,5.000000,5.0,5.0,6,0.036000,0.226833,0.737167
AVpfl8cLLJeJML43AE3S,6619,4.671098,1.0,5.0,6619,0.025188,0.425041,0.549771


### 2. ¿Qué productos deben ser descartartados?
Productos que tienen una valoración media inferior a 3.5 o una mayor proporción de opiniones negativas en comparación con las positivas (para tener en cuenta las opiniones e incluir los casos en los que no hay valoración). Esto puede indicar una cantidad desproporcionada de opiniones en relación con las ventas o el compromiso con el producto.

In [ ]:
product_dropped = statistics_df[
    (statistics_df['reviews.rating_mean'] < 3.5) | (statistics_df['neg_mean'] > statistics_df['pos_mean'])]

product_dropped

,id_count,reviews.rating_mean,reviews.rating_min,reviews.rating_max,reviews.text_count,neg_mean,pos_mean,neu_mean
id,,,,,,,,
AVpf_4sUilAPnD_xlwYV,15,3.066667,1.0,5.0,15,0.075933,0.310467,0.613667
AVpfiBlyLJeJML43-4Tp,13,2.461538,1.0,5.0,13,0.081615,0.224769,0.693692
AVzvXXwEvKc47QAVfRhx,8,3.125000,1.0,5.0,8,0.104250,0.164625,0.731250


### 3. ¿Qué productos son basura?
Los productos que no tienen ventas o tienen menos o igual a 15 ventas, lo que indica una baja demanda, y tienen una puntuación media de 2.5 o inferior, lo que sugiere una mala calidad o satisfacción del producto. Hay casos en los que el cliente ha dejado una opinión pero no una valoración, así que para incluir estos casos también hemos puesto la condición de que los productos que tienen una puntuación negativa mayor que la positiva también son basura.

In [ ]:
product_junked = statistics_df[
    (statistics_df['id_count'] <= 15) &
    ((statistics_df['reviews.rating_mean'] <= 2.5) | (statistics_df['neg_mean'] > statistics_df['pos_mean']))]

product_junked

,id_count,reviews.rating_mean,reviews.rating_min,reviews.rating_max,reviews.text_count,neg_mean,pos_mean,neu_mean
id,,,,,,,,
AVpfiBlyLJeJML43-4Tp,13,2.461538,1.0,5.0,13,0.081615,0.224769,0.693692


### 4. ¿Qué producto debería ser recomendado al cliente?
El producto que tenga una calificación media de 4.8 o superior, lo que indica una calidad excepcional y satisfacción, y que demuestre un sentimiento positivo superior al sentimiento neutral, lo que sugiere una retroalimentación abrumadoramente positiva por parte de los clientes.

In [ ]:
product_recommended = statistics_df[
    (statistics_df['reviews.rating_mean'] >= 4.8) &
     ((statistics_df['pos_mean'] > statistics_df['neu_mean']))]

product_recommended

,id_count,reviews.rating_mean,reviews.rating_min,reviews.rating_max,reviews.text_count,neg_mean,pos_mean,neu_mean
id,,,,,,,,
AVph0EeEilAPnD_x9myq,6,4.833333,4.0,5.0,6,0.053333,0.5185,0.428167


### 5. ¿Cuáles son los mejores productos para los consumidores?

Todos los productos que tengan una calificación media mayor o igual a 4.5, lo que indica que los clientes se han tomado el tiempo para calificar el producto y lo han calificado muy alto para mostrar su experiencia muy positiva. Además de esto, para incluir a los clientes que han revisado el producto pero no han dejado una calificación y tener en cuenta las reseñas en general, queremos que el sentimiento general sea más positivo que negativo. Hasta ahora, esto nos ha dado productos altamente calificados, pero para este caso queremos los mejores productos para los consumidores y por eso también filtraremos por los productos más vendidos, buscando productos que se hayan comprado más de 1000 veces.

In [ ]:
product_best = statistics_df[((statistics_df['reviews.rating_mean'] >= 4.5) | statistics_df['pos_mean'] > 0.4) &
                              (statistics_df['pos_mean'] > statistics_df['neg_mean']) &
                              (statistics_df['id_count'] > 1000)]

product_best

,id_count,reviews.rating_mean,reviews.rating_min,reviews.rating_max,reviews.text_count,neg_mean,pos_mean,neu_mean
id,,,,,,,,
AV1YE_muvKc47QAVgpwE,5056,4.707278,1.0,5.0,5056,0.072376,0.397614,0.530009
AV1YnRtnglJLPUi8IJmV,3176,4.772355,1.0,5.0,3176,0.036899,0.408558,0.554542
AVpfl8cLLJeJML43AE3S,6619,4.671098,1.0,5.0,6619,0.025188,0.425041,0.549771
AVphgVaX1cnluZ0-DR74,10966,4.454222,1.0,5.0,10966,0.045242,0.426217,0.528449
AVpjEN4jLJeJML43rpUe,1038,4.504817,1.0,5.0,1038,0.051052,0.411398,0.537543
AVqVGWLKnnc1JgDc3jF1,1685,4.527003,1.0,5.0,1685,0.039983,0.437753,0.522265
AVqkIhwDv8e3D1O-lebb,2814,4.586709,1.0,5.0,2814,0.048247,0.421815,0.529936


### 6. ¿Qué productos deberían ser planeados para el inventario del próximo invierno?

Todos los productos que han sido altamente calificados y comprados con frecuencia durante los meses de noviembre a febrero se consideran productos que deben ser altamente considerados para el inventario del invierno.

In [ ]:
winter_df = df_sentiments[df_sentiments['month'].isin([11, 12, 1, 2])]
winter_group = winter_df.groupby('id').agg({'id': ['count'], 'reviews.rating': ['mean'], 'neg': ['mean'], 'pos': ['mean'], 'neu': ['mean']})
winter_group.columns = ['_'.join(col).strip() for col in winter_group.columns.values]

In [ ]:
winter_inventory = winter_group[((winter_group['reviews.rating_mean'] >= 4) | (winter_group['pos_mean'] > 0.4))  &
                                (winter_group['id_count'] >= 1000) & (winter_group['pos_mean'] > winter_group['neg_mean'])]

winter_inventory

,id_count,reviews.rating_mean,neg_mean,pos_mean,neu_mean
id,,,,,
AV1YE_muvKc47QAVgpwE,1740,4.707471,0.071898,0.398493,0.529598
AV1YnRtnglJLPUi8IJmV,1197,4.748538,0.036383,0.405778,0.557838
AVpfl8cLLJeJML43AE3S,2412,4.675788,0.024687,0.435254,0.540067
AVphgVaX1cnluZ0-DR74,6904,4.459444,0.042583,0.431492,0.525923
AVqVGWLKnnc1JgDc3jF1,1044,4.526820,0.038419,0.442791,0.518785
AVqkIhwDv8e3D1O-lebb,2073,4.597202,0.047907,0.425990,0.526101


### 7.¿Qué productos requieren publicidad?

Productos que no han sido comprados con frecuencia, o que no han tenido altas ventas, pero que sin embargo han sido altamente calificados y en general tienen una reseña con un sentimiento positivo.

In [ ]:
product_advertis = statistics_df[(statistics_df['pos_mean'] > statistics_df['neg_mean']) &
                         (statistics_df['id_count'] <= 50) & (statistics_df['reviews.rating_mean'] >= 4.5)]

product_advertis

,id_count,reviews.rating_mean,reviews.rating_min,reviews.rating_max,reviews.text_count,neg_mean,pos_mean,neu_mean
id,,,,,,,,
AVpe7AsMilAPnD_xQ78G,19,4.666667,4.0,5.0,19,0.055000,0.247737,0.697316
AVpff7_VilAPnD_xc1E_,6,5.000000,5.0,5.0,6,0.036000,0.226833,0.737167
AVpgdkC8ilAPnD_xsvyi,10,4.700000,4.0,5.0,10,0.000000,0.527500,0.472500
AVpggqsrLJeJML43O5zp,8,4.875000,4.0,5.0,8,0.102750,0.274750,0.622750
AVph0EeEilAPnD_x9myq,6,4.833333,4.0,5.0,6,0.053333,0.518500,0.428167
AVpioXbb1cnluZ0-PImd,2,4.500000,4.0,5.0,2,0.093000,0.281000,0.626000
AVqVGZN9QMlgsOJE6eUZ,6,4.666667,4.0,5.0,6,0.038167,0.368000,0.593833
AVqkIdntQMlgsOJE6fuB,30,4.600000,3.0,5.0,30,0.058967,0.326967,0.614033
AVqkIhxunnc1JgDc3kg_,7,5.000000,5.0,5.0,7,0.047571,0.376857,0.575857
